In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pymc3 as pm



sns.set()


###### must run scb_population_deaths before this #####
### 2020 population 2019 * growth factor 2018-2019

In [ ]:
##### MUST RUN covid_confirmed and then PYMC3_mortality to produce ifr_data.pkl

df = pd.read_pickle('ifr_data.pkl')

In [ ]:
df.tail()

In [ ]:
swe = df.loc[df['country'] == 'Sweden']
swe

In [ ]:
deaths_per_m = pd.read_pickle('deaths_per_m_1990-2020-YTD.pkl')

In [ ]:
deaths_per_m

In [ ]:
month_theta_map = dict(
    zip(
        deaths_per_m.columns,
        np.radians(np.arange(0,len(deaths_per_m.columns) + 0)) / 12 * 360))

month_theta_map

In [ ]:
def month_to_theta(month) :
    return month_theta_map[month]

month_to_theta('Dec')

In [ ]:
plt.style.available

In [ ]:
last_complete_month = "Oct"
last_complete_month_num = 10

plt.style.use('arviz-darkgrid')
#sns.set_style('dark')


def plot(year,color):
    
    if year != 2020:
    
        plt.polar(month_theta_map.values(),deaths_per_m.loc[year,:],'o--',
                  color=color,label='{}'.format(year))
        
    else:
        
        plt.polar(list(month_theta_map.values())[:last_complete_month_num],
                  deaths_per_m.loc[year,:last_complete_month],'o--',
                  color=color,label='{}'.format(year))
    
    ax = plt.gca()

    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    
    ax.set_thetagrids((np.arange(0,360,30)))
    ax.set_xticklabels(deaths_per_m.columns)
    
    ax.set_facecolor('silver')


    plt.legend(loc='upper left')

In [ ]:
def plot_cum(year,color):
    
    if year != 2020:
    
        plt.polar(month_theta_map.values(),deaths_per_m.loc[year,:].cumsum(),'o--',
                  color=color,label='{}'.format(year))
        
    else:
        
        plt.polar(list(month_theta_map.values())[:last_complete_month_num],
                  deaths_per_m.loc[year,:last_complete_month].cumsum(),'o--',
                  color=color,label='{}'.format(year))
    
    ax = plt.gca()

    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    
    ax.set_thetagrids((np.arange(0,360,30)))
    ax.set_xticklabels(deaths_per_m.columns)
    
    ax.set_facecolor('silver')
    

    plt.legend(loc='upper left')

In [ ]:
colors = ['orange','lime','yellow','k','m','crimson']


plt.figure(figsize=(12,12))

plt.subplot(1,1,1,projection='polar')
plt.title('SWEDEN monthly deaths per million, 2015-2020',color='crimson')

years = deaths_per_m.index

for i,y in enumerate(years[-6:]):
    
    plot(y,colors[i])
    
plt.savefig('Sweden_deaths_polar_plot.jpg',format='jpg')



In [ ]:
plt.figure(figsize=(12,12))

plt.subplot(1,1,1,projection='polar')
plt.title('SWEDEN cumulative monthly deaths per million, 2015-2020',color='crimson')


for i,y in enumerate(years[-6:]):
    
    plot_cum(y,colors[i])
    
plt.savefig('sweden_cum_deaths_polar_plot.jpg',format='jpg')

In [ ]:
last_complete_month

In [ ]:
years_2015_2020 = deaths_per_m.loc[2015 : 2020]
years_2015_2020

In [ ]:
years_2015_2018_monthly_mean = years_2015_2020.loc[2015 : 2018].mean()
years_2015_2018_monthly_mean

In [ ]:
excess = years_2015_2020.loc[2020] - years_2015_2018_monthly_mean
excess

In [ ]:
def plot_excess(r):
    
    plt.polar(list(month_theta_map.values())[:last_complete_month_num],r,'o--',
             color='orange',label='excess deaths per million')
    
    ax = plt.gca()

    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    
    ax.set_thetagrids((np.arange(0,360,30)))
    ax.set_xticklabels(deaths_per_m.columns)
    
    ax.set_facecolor('silver')

    
    circle = np.linspace(0,2*np.pi,100)
    
    plt.polar(circle,np.repeat(0,len(circle)),color='crimson',label='zero')
    plt.legend(loc='upper left')

In [ ]:
plt.figure(figsize=(18,12))
plot_excess(excess[:last_complete_month_num])
plt.title('SWEDEN monthly excess deaths per million [baseline 2015-2018]',color='crimson')
plt.savefig('sweden_polar_excess.jpg',format='jpg')

In [ ]:
excess[:last_complete_month_num].cumsum()

In [ ]:

plt.figure(figsize=(18,12))

plot_excess(excess[:last_complete_month_num].cumsum())
plt.title('SWEDEN cumulative monthly excess deaths per million [baseline 2015-2018]',color='crimson')
plt.savefig('sweden_polar_cum_excess.jpg',format='jpg')

In [ ]:
def parse_scb_prel(YTD=True):
    
    def monthname2number(m):
        if m == 'januari' : return 1
        if m == 'februari' : return 2
        if m == 'mars' : return 3
        if m == 'april' : return 4
        if m == 'maj' : return 5
        if m == 'juni' : return 6
        if m == 'juli' : return 7
        if m == 'augusti' : return 8
        if m == 'september' : return 9
        if m == 'oktober' : return 10
        if m == 'november' : return 11
        if m == 'december' : return 12

    df = pd.read_csv('scb-dead_per_day.csv',sep=';',usecols=[1,2,3,4,5,6,7,10,11],thousands=',')
    
    
    df['month'] = df['Månad'].apply(monthname2number)
    df[['Månad','month']] = df[['Månad','month']].replace(np.nan,0)
    df['month'] = df['month'].astype(int)
    
    
    df = df.replace(np.nan,0)

    
    df[['2015','2016','2017','2018','2019','2020','2015-2019']] = df[['2015','2016','2017','2018','2019','2020','2015-2019']].astype(int)

    unknown_death_day = df.iloc[-1,:7]
    
    df.drop(366,inplace=True)
    
    if YTD:
        df = df.loc[df['2020'] != 0]
        
    df = df.drop('Månad',axis=1)
    
    known_death_day = df.iloc[:,:7].sum()
    
    year_fraction = len(df) / 366
    
    unknown_death_day[['2015','2016','2017','2018','2019','2015-2019']] = unknown_death_day[
    ['2015','2016','2017','2018','2019','2015-2019']] * year_fraction
    
    dead_pct_per_day = df[['2015','2016','2017','2018','2019','2020','2015-2019']] / known_death_day
    
    all_deaths = df[['2015','2016','2017','2018','2019','2020','2015-2019']] + unknown_death_day * dead_pct_per_day
    
    if YTD:
        index = pd.date_range('2020-01-01',periods=len(all_deaths))
    else:
        index = pd.date_range('2020-01-01','2020-12-31')
        
    all_deaths.index = index
    
    return all_deaths
    

In [ ]:
df_scb = parse_scb_prel(YTD=False)
df_scb

In [ ]:
pop = pd.read_csv('scb_population.csv',sep=',')
pop

In [ ]:
pop_growth_factor = pop['2019'] / pop['2018']
pop['2020'] = pop['2019'] * pop_growth_factor
pop = pop.loc[:,'2015' : '2020'].T
pop = pd.Series(pop[0])
pop

In [ ]:
last_date = df_scb.loc[df_scb['2020'] == 0].index[0] - pd.DateOffset(weeks=1)
#df_scb = df_scb[:last_date]
df_scb.drop('2015-2019',axis=1,inplace=True)
df_scb

In [ ]:
df_scb_per_m = df_scb / (pop / 1e6)
df_scb_per_m

In [ ]:
df_scb_weekly = df_scb_per_m.resample('W-Tue').sum()

df_scb_weekly = df_scb_weekly[:-1]

In [ ]:
week_theta_map = dict(
    zip(
        df_scb_weekly.index.date,
        np.radians(np.arange(0,len(df_scb_weekly.index) + 0)) / len(df_scb_weekly.index) * 360))

week_theta_map

In [ ]:
xticks_month = df_scb_weekly.index.month
xticks_day = df_scb_weekly.index.day

xticks = list(zip(xticks_month,xticks_day))

xticks = [str(x[0]) + '/' + str(x[1]) for x in xticks]


In [ ]:
np.array(list(week_theta_map.values()))

len_2020 = len(df_scb_weekly[:last_date])
max_ = df_scb_weekly.max().max()

In [ ]:


def plot_weekly(series,color,label,max_,title):


    
    if label != '2020':

        plt.polar(week_theta_map.values(),series,'o--',color=color,label=label)

    else:
        plt.polar(np.array(list(week_theta_map.values()))[:len_2020],
                  series[:last_date],'o--',color=color,label=label)
        
        plt.polar(np.array(list(week_theta_map.values()))[len_2020 -1 ],max_,'X',
                  color='crimson',markersize=10,label='latest 2020 data')
        
    ax = plt.gca()
    
    ax.set_title(title)

    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    
    ax.set_thetagrids(np.arange(0,360,360 / len(df_scb_weekly)))
    ax.set_xticklabels(xticks,rotation=180)

    
    ax.set_facecolor('silver')
    ax.legend(loc='upper left')
    



In [ ]:
plt.figure(figsize=(18,12))

title = 'SWEDEN weekly deaths per million, 2020 YTD vs 2015-2019'

plot_weekly(df_scb_weekly['2019'],'g','2019',max_,title)
plot_weekly(df_scb_weekly['2018'],'b','2018',max_,title)
plot_weekly(df_scb_weekly['2017'],'orange','2017',max_,title)
plot_weekly(df_scb_weekly['2016'],'lime','2016',max_,title)
plot_weekly(df_scb_weekly['2015'],'k','2015',max_,title)
plot_weekly(df_scb_weekly['2020'],'r','2020',max_,title)

plt.savefig('sweden_polar_weekly.jpg',format='jpg')


In [ ]:
max_ = df_scb_weekly.cumsum().max().max()

In [ ]:
title = 'SWEDEN weekly cumulative deaths per million, 2020 YTD vs 2015-2019'


plt.figure(figsize=(18,12))
plot_weekly(df_scb_weekly['2019'].cumsum(),'g','2019',max_,title)
plot_weekly(df_scb_weekly['2018'].cumsum(),'b','2018',max_,title)
plot_weekly(df_scb_weekly['2017'].cumsum(),'orange','2017',max_,title)
plot_weekly(df_scb_weekly['2016'].cumsum(),'lime','2016',max_,title)
plot_weekly(df_scb_weekly['2015'].cumsum(),'k','2015',max_,title)
plot_weekly(df_scb_weekly['2020'].cumsum(),'r','2020',max_,title)

plt.savefig('sweden_polar_cum_weekly.jpg',format='jpg')

